In [13]:
# import pandas + plotly theme
import pandas as pd
from utils.plotly_theme import *

In [14]:
# test interpolation function (for cap,cop,plr)
from utils.interp import interp_vector

fp = [3.815, 3.955, 4.133, 3.74] # cop
xp = [1_465_000, 1_098_750, 732_500, 366_250] # cap

fp_1 = [3.74, 4.133, 3.955, 3.815] # cop
xp_1 = [366_250, 732_500, 1_098_750, 1_465_000] # cap

x = [1_200_000] # 1_465_200

print(interp_vector(xp, fp, x))
print(interp_vector(xp_1, fp_1, x))

[3.74]
[3.91629693]


In [15]:
# Example usage of the decarb-tool

from src.metadata import Metadata, EmissionScenario
from utils.inputs import get_city_and_zone

# Load location data
location_data = pd.read_csv("data/input/locations.csv")
zip_code = "94709" # Berkeley, CA -> will be replaced by user input from UI (search by zip-code or city name)
locations = get_city_and_zone(zip_code, location_data)

# Based on zip-code, SET location, climate_zone, time_zone
location = locations.city
ashrae_climate_zone = locations.ashrae_climate_zone
time_zone = locations.time_zone
# gea_grid_region = locations.gea_grid_region # not yet implemented

# Set building type, building vintage, load type and equipment via equipment scenario -> eventually replaced by user input from UI
building_type = "OfficeLarge" # currently we have "Hospital", "OfficeLarge", "OutPatientHealthCare"
vintage = 2022 # currently we have 2004, 2022
load_type = "load_simulated"
equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"] # list of equipment scenarios to compare

settings = Metadata.create(
    location=location,
    ashrae_climate_zone=ashrae_climate_zone,
    time_zone=time_zone,
    building_type=building_type,
    vintage=vintage,
    load_type=load_type,
    equipment_scenarios=equipment_scenarios,
)

In [16]:
# check it emission scenarios are loaded correctly

scenario = settings.get_emission_scenario("em_scenario_a")
print(scenario.year)

settings.model_dump()

2025


{'location': 'Berkeley',
 'building_type': 'OfficeLarge',
 'vintage': 2022,
 'load_type': 'load_simulated',
 'ashrae_climate_zone': '3C',
 'equipment_scenarios': ['eq_scenario_1',
  'eq_scenario_2',
  'eq_scenario_3',
  'eq_scenario_4',
  'eq_scenario_5'],
 'emission_settings': [{'em_scen_id': 'em_scenario_a',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_type': 'Combustion only',
   'shortrun_weighting': 1.0,
   'annual_refrig_leakage': 0.01,
   'annual_ng_leakage': 0.005,
   'year': 2025},
  {'em_scen_id': 'em_scenario_b',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_type': 'Combustion only',
   'shortrun_weighting': 0.5,
   'annual_refrig_leakage': 0.01,
   'annual_ng_leakage': 0.005,
   'year': 2035},
  {'em_scen_id': 'em_scenario_c',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_

In [17]:
# test equipment library load

from src.equipment import load_library
equipment_library = load_library("data/input/equipment_data.json").model_dump()

equipment_library

{'equipment': [{'eq_id': 'hr01',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water',
   'eq_manufacturer': None,
   'model': 'GenericHR_WWHP',
   'fuel': 'electricity',
   'capacity_W': None,
   'performance': {'cop_curve': None,
    'cap_curve': None,
    'plr_curve': {'capacity_W': [146500.0, 109875.0, 73250.0, 36625.0],
     'cop': [3.815, 3.955, 4.133, 3.74]},
    'efficiency': None,
    'constraints': None},
   'emissions': None},
  {'eq_id': 'hr02',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water_scroll',
   'eq_manufacturer': None,
   'model': 'IMC-WWHP-30Ton-120HHWST',
   'fuel': 'electricity',
   'capacity_W': None,
   'performance': {'cop_curve': None,
    'cap_curve': None,
    'plr_curve': {'capacity_W': [105500.0, 50000.0], 'cop': [3.68, 4.2]},
    'efficiency': None,
    'constraints': None},
   'emissions': None},
  {'eq_id': 'hr03',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water_scroll',
   'eq_manufacturer': None,
   'model':

In [18]:
# check library in normalized dataframe format

equipment_data = pd.json_normalize(equipment_library["equipment"])
equipment_data.head(50)

,eq_id,eq_type,eq_subtype,eq_manufacturer,model,fuel,capacity_W,emissions,performance.cop_curve,performance.cap_curve,performance.plr_curve.capacity_W,performance.plr_curve.cop,performance.efficiency,performance.constraints,performance.cop_curve.t_out_C,performance.cop_curve.cop,performance.cap_curve.t_out_C,performance.cap_curve.capacity_W,performance.plr_curve,emissions.co2_kg_per_mwh
0,hr01,hr_heat_pump,water_to_water,None,GenericHR_WWHP,electricity,NaN,NaN,NaN,NaN,"[146500.0, 109875.0, 73250.0, 36625.0]","[3.815, 3.955, 4.133, 3.74]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,hr02,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-30Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[105500.0, 50000.0]","[3.68, 4.2]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
2,hr03,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-30Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[105500.0, 50000.0]","[2.68, 3.3]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
3,hr04,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-60Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[211000.0, 140000.0, 70000.0]","[3.68, 4.0, 4.2]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
4,hr05,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-60Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[211000.0, 140000.0, 70000.0]","[2.68, 3.1, 3.3]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
5,hr06,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-250Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[879250.0, 700000.0, 500000.0, 300000.0, 17500...","[3.68, 3.9, 4.3, 4.5, 4.6]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
6,hr07,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-250Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[879250.0, 700000.0, 500000.0, 300000.0, 17500...","[2.68, 2.9, 3.3, 3.5, 3.6]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
7,hr08,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-500Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[1758500.0, 1200000.0, 800000.0, 600000.0, 350...","[3.96, 4.2, 4.6, 4.8, 5.0]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
8,hr09,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-500Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[1758500.0, 1200000.0, 800000.0, 600000.0, 350...","[2.97, 3.2, 3.6, 3.8, 4.0]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
9,hp01,heat_pump,air_to_water,None,GenericAWHP,electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"[-10.0, 0.0, 10.0, 20.0, 30.0]","[2.0, 2.5, 3.2, 3.8, 4.2]","[-10.0, 0.0, 10.0, 20.0, 30.0]","[100000.0, 120000.0, 140000.0, 160000.0, 18000...",NaN,NaN


In [19]:
# test adding a new emission scenario

settings.add_emission_scenario(
    EmissionScenario(
        em_scen_id="new_007",
        grid_scenario="MidCase",
        gea_grid_region="CAISO",
        time_zone="America/New_York",
        emission_type="Combustion only",
        annual_ng_leakage=2.0,
        annual_refrig_leakage=5.0,
        shortrun_weighting=1.0,
        year=2030,
    )
)

print(settings["new_007"].year)
print(settings["new_007"].grid_scenario)

x = settings.__getitem__("new_007")
print(x.gea_grid_region)


2030
MidCase
CAISO


In [20]:

# check if emission scenario was added correctly
from pprint import pprint

pprint(settings.dict())

{'ashrae_climate_zone': '3C',
 'building_type': 'OfficeLarge',
 'emission_settings': [{'annual_ng_leakage': 0.005,
                        'annual_refrig_leakage': 0.01,
                        'em_scen_id': 'em_scenario_a',
                        'emission_type': 'Combustion only',
                        'gea_grid_region': 'CAISO',
                        'grid_scenario': 'MidCase',
                        'shortrun_weighting': 1.0,
                        'time_zone': 'America/Los_Angeles',
                        'year': 2025},
                       {'annual_ng_leakage': 0.005,
                        'annual_refrig_leakage': 0.01,
                        'em_scen_id': 'em_scenario_b',
                        'emission_type': 'Combustion only',
                        'gea_grid_region': 'CAISO',
                        'grid_scenario': 'MidCase',
                        'shortrun_weighting': 0.5,
                        'time_zone': 'America/Los_Angeles',
                        

/var/folders/x4/3lxftn1s4zgf32y4j4tqx4kh0000gn/T/ipykernel_51165/3253440217.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(settings.dict())


In [21]:
# Get datasets based on input (location, load_type) saved in settings object
from src.loads import get_load_data
from src.emissions import get_emissions_data

# get load data
load_data = get_load_data(settings)

# get emission data
emissions_data = get_emissions_data(settings["em_scenario_a"])

# inspect load data
load_data.df.head(30)

/Users/toby/Desktop/Repositories/decarb-tool/src/loads.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=False)


,t_out_C,heating_W,cooling_W
timestamp,,,
2001-01-01 01:00:00,8.000000,0.000000,0.000000
2001-01-01 02:00:00,8.000000,0.000000,0.000000
2001-01-01 03:00:00,8.583333,0.000000,0.000000
2001-01-01 04:00:00,9.000000,0.000000,0.000000
2001-01-01 05:00:00,9.000000,0.000000,0.000000
2001-01-01 06:00:00,8.416667,0.000000,0.000000
2001-01-01 07:00:00,8.583333,0.000000,0.000000
2001-01-01 08:00:00,9.583333,0.000000,0.000000
2001-01-01 09:00:00,11.166667,0.000000,0.000000


In [22]:
# inspect emissions data
emissions_data.df.head(30)

,em_scen_id,emission_scenario,gea_grid_region,time_zone,emission_type,year,lrmer_co2e_c,lrmer_co2e_p,srmer_co2e_c,srmer_co2e_p
timestamp,,,,,,,,,,
2025-01-01 00:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,408.0,137.7,561.2,173.2
2025-01-01 01:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,431.1,142.3,547.6,163.8
2025-01-01 02:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,429.1,147.9,461.2,162.2
2025-01-01 03:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,413.6,149.6,480.7,167.0
2025-01-01 04:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,417.8,145.3,480.9,179.3
2025-01-01 05:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,396.4,137.6,533.5,174.0
2025-01-01 06:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,364.3,126.7,546.2,177.9
2025-01-01 07:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,269.0,91.7,656.7,154.8
2025-01-01 08:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,139.6,46.7,554.9,158.2


In [23]:
# Test-loading individual pieces from equipment library (not changing the actual sim settings)

from src.equipment import load_library

library = load_library("data/input/equipment_data.json")

hp_a = library.get_equipment("hp01")
print(hp_a)

scenario_a = library.get_scenario("eq_scenario_1")
print(scenario_a)

eq_id='hp01' eq_type='heat_pump' eq_subtype='air_to_water' eq_manufacturer=None model='GenericAWHP' fuel='electricity' capacity_W=None performance=Performance(cop_curve=COPCurve(t_out_C=[-10.0, 0.0, 10.0, 20.0, 30.0], cop=[2.0, 2.5, 3.2, 3.8, 4.2]), cap_curve=CapCurve(t_out_C=[-10.0, 0.0, 10.0, 20.0, 30.0], capacity_W=[100000.0, 120000.0, 140000.0, 160000.0, 180000.0]), plr_curve=None, efficiency=None, constraints=None) emissions=None
eq_scen_id='eq_scenario_1' eq_scen_name='Baseline System' hr_wwhp=None awhp_h='hp01' awhp_c='hp01' awhp_sizing=1.0 boiler='bo01' chiller='ch01' resistance_heater='res01'


In [24]:
# calculate site energy for all equipment scenarios in settings object

from src.energy import loads_to_site_energy

site_energy = loads_to_site_energy(load_data, library, settings.equipment_scenarios, detail=True)

print(f"Equipment scenarios considered: {len(settings.equipment_scenarios)}")
print(f"Computed number of years: {len(site_energy) / 8760}")

site_energy.head(30)


Equipment scenarios considered: 5
Computed number of years: 5.0


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,awhp_cop_c,awhp_chw_W,elec_awhp_c_Wh,chiller_cop,chiller_chw_W,elec_chiller_Wh,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-01 01:00:00,8.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.06,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 02:00:00,8.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.06,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 03:00:00,8.58,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.10,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 04:00:00,9.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.13,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 05:00:00,9.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.13,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 06:00:00,8.42,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.09,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 07:00:00,8.58,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.10,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 08:00:00,9.58,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.17,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN
2001-01-01 09:00:00,11.17,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,3.27,0.00,0.00,4.0,0.0,0.0,eq_scenario_1,NaN,NaN,NaN


In [25]:
# check available emission scenarios before running site to source
settings.list_emission_scenarios()

['em_scenario_a', 'em_scenario_b', 'em_scenario_c', 'new_007']

In [26]:
# run site to source

from src.energy import site_to_source

source_energy = site_to_source(site_energy, metadata=settings)

print(f"Equipment scenarios considered: {len(settings.equipment_scenarios)}")
print(f"Emission scenarios considered: {len(settings.emission_settings)}")
print(f"Computed number of years: {len(source_energy) / 8760}")

source_energy.head(30)

Equipment scenarios considered: 5
Emission scenarios considered: 4
Computed number of years: 20.0


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W,elec_emissions_rate,year,elec_emissions,gas_emissions,total_refrig_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,8.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,489.35,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 02:00:00,8.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,445.15,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 03:00:00,8.58,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,447.15,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 04:00:00,9.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,449.35,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 05:00:00,9.00,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,464.95,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 06:00:00,8.42,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,455.25,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 07:00:00,8.58,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,462.85,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 08:00:00,9.58,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,347.25,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 09:00:00,11.17,0.00,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,292.30,2025,0.000000,0.0,0.0,em_scenario_a


In [27]:
df = source_energy.copy()

df

,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W,elec_emissions_rate,year,elec_emissions,gas_emissions,total_refrig_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,8.00,0.00,0.0,0.00,0.0,0.00,0.0,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,489.35,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 02:00:00,8.00,0.00,0.0,0.00,0.0,0.00,0.0,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,445.15,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 03:00:00,8.58,0.00,0.0,0.00,0.0,0.00,0.0,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,447.15,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 04:00:00,9.00,0.00,0.0,0.00,0.0,0.00,0.0,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,449.35,2025,0.000000,0.0,0.0,em_scenario_a
2025-01-01 05:00:00,9.00,0.00,0.0,0.00,0.0,0.00,0.0,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,464.95,2025,0.000000,0.0,0.0,em_scenario_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 20:00:00,10.25,14494.75,0.0,5426.71,0.0,14494.75,0.0,NaN,NaN,NaN,...,eq_scenario_5,NaN,NaN,NaN,423.20,2030,2.296584,0.0,0.0,new_007
2030-12-31 21:00:00,10.75,9324.24,0.0,3462.40,0.0,9324.24,0.0,NaN,NaN,NaN,...,eq_scenario_5,NaN,NaN,NaN,427.80,2030,1.481215,0.0,0.0,new_007
2030-12-31 22:00:00,12.00,885.29,0.0,322.16,0.0,885.29,0.0,NaN,NaN,NaN,...,eq_scenario_5,NaN,NaN,NaN,425.15,2030,0.136966,0.0,0.0,new_007


In [28]:
from src.visuals import plot_energy_and_emissions

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"]
emission_scenarios = ["new_007"]

plot_energy_and_emissions(df, equipment_scenarios, emission_scenarios, unit_mode="SI")

In [29]:
from src.visuals import plot_emission_scenarios_grouped

df = source_energy.copy()

emission_scenarios = ['em_scenario_a', 'em_scenario_b', 'em_scenario_c']

plot_emission_scenarios_grouped(df, equipment_scenarios, emission_scenarios, unit_mode="SI")

In [30]:
from src.visuals import plot_meter_timeseries

df = source_energy.copy()

equipment_scenarios = "eq_scenario_1"
emission_scenarios = "em_scenario_a"

plot_meter_timeseries(df, equipment_scenario=equipment_scenarios, emission_scenario=emission_scenarios, stacked=True, include_gas=True, unit_mode="IP")

In [31]:
from src.visuals import plot_emissions_heatmap

df = source_energy.copy()

equipment_scenarios = "eq_scenario_1"
emission_scenarios = "em_scenario_a"

plot_emissions_heatmap(df, equipment_scenarios, emission_scenarios, unit_mode="SI", emission_type="elec_emissions")

In [32]:
#! Update for mutliple scenarios and years

from src.visuals import plot_energy_breakdown

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2"]
emission_scenarios = ["em_scenario_a"]

plot_energy_breakdown(df, equipment_scenarios, emission_scenarios)
